In [1]:
import os 
new_directory = "C:\\Users\\SAMSUNG\\OneDrive\\Desktop"
os.chdir(new_directory)
print("New working directory:", os.getcwd())


New working directory: C:\Users\SAMSUNG\OneDrive\Desktop


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.bbc.com/news/articles/cwy04km1zk0o"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

title = soup.find('h1')
print("Title:", title.text.strip() if title else "No title found")

paragraphs = soup.find_all('p')
article_text = "\n".join(p.text for p in paragraphs)

print("\nArticle Text:\n", article_text)


Title: Israel security cabinet approves plan to 'capture' Gaza, official says

Article Text:
 Israel's security cabinet has approved a plan to expand its military offensive against Hamas which includes the "capture" of Gaza and the holding of its territory, according to an Israeli official.
Prime Minister Benjamin Netanyahu said the cabinet had decided on a "forceful operation" to destroy Hamas and rescue its remaining hostages, and that Gaza's 2.1 million population "will be moved, to protect it".
He did not say how much territory would be seized by troops, but he stressed that "they will not enter and come out".
The cabinet also approved, in principle, a plan to deliver aid through private companies, which would end a two-month blockade the UN says has caused severe food shortages.
The UN and other aid agencies have said the proposal would be a breach of basic humanitarian principles and that they will not co-operate.
A Hamas official said the group rejected Israel's "pressure and bl

In [7]:
#for For BBC, CNN, NYT, NPR
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_article(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        title = soup.find('h1').text.strip()
        paragraphs = soup.find_all('p')
        content = " ".join(p.text for p in paragraphs)
        
        return {'url': url, 'title': title, 'content': content}
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# list of URLs
urls = [
    "https://www.bbc.com/news/articles/c367g52x7l6o"
]

data = []
for url in urls:
    article = scrape_article(url)
    if article:
        data.append(article)

df = pd.DataFrame(data)
df.to_csv('real_news.csv', index=False)


In [8]:
#for the guardian
import requests
import pandas as pd

api_key = "736b97fc-3458-480d-9a7e-42b8dac30a05"
url = f"https://content.guardianapis.com/search?q=politics&api-key={api_key}&show-fields=all&page=1"

response = requests.get(url)
data = response.json()
articles = data['response']['results']

records = []
for a in articles:
    records.append({
        "title": a['webTitle'],
        "date": a['webPublicationDate'],
        "url": a['webUrl'],
        "section": a['sectionName']
    })

df = pd.DataFrame(records)
df.to_csv("guardian_articles.csv", index=False)


In [9]:
#BBC World 
import requests
from bs4 import BeautifulSoup

def get_bbc_article_urls(section_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(section_url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    links = set()
    for a in soup.find_all('a', href=True):
        href = a['href']
        if href.startswith('/news') and '/live/' not in href:
            links.add('https://www.bbc.com' + href.split('?')[0])
    
    return list(links)

urls = get_bbc_article_urls('https://www.bbc.com/news/world')
for url in urls:
    print(url)


https://www.bbc.com/news/articles/c0qz32wzeego
https://www.bbc.com/news/bbcverify
https://www.bbc.com/news/bbcindepth
https://www.bbc.com/news/articles/c0qnvnxn0y7o
https://www.bbc.com/news/articles/c3v9gnd0xr0o
https://www.bbc.com/news/articles/cd6j8ew1w0qo
https://www.bbc.com/news/war-in-ukraine
https://www.bbc.com/news/uk
https://www.bbc.com/news/world/latin_america
https://www.bbc.com/news/articles/cm24l14l4zmo
https://www.bbc.com/news/world/asia/india
https://www.bbc.com/news/in_pictures
https://www.bbc.com/news/northern_ireland/northern_ireland_politics
https://www.bbc.com/news/articles/ckg2ekjjgldo
https://www.bbc.com/news/articles/c4g2n07nngvo
https://www.bbc.com/news/articles/cze17n02gego
https://www.bbc.com/news/articles/c93g0wp8vqdo
https://www.bbc.com/news/world/middle_east
https://www.bbc.com/news/articles/ckg2vyxzg3qo
https://www.bbc.com/news/northern_ireland
https://www.bbc.com/news/wales/wales_politics
https://www.bbc.com/news/videos/crrz4lred45o
https://www.bbc.com/new

In [17]:
#CNN World 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def get_cnn_article_urls():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)

    driver.get("https://edition.cnn.com/world")
    time.sleep(5)  # Let the JS load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    article_urls = set()
    for tag in soup.select('a[href^="/"]'):
        href = tag['href']
        if href.startswith('/202') and href.endswith('.html'):
            full_url = 'https://edition.cnn.com' + href
            article_urls.add(full_url)

    return list(article_urls)

cnn_urls = get_cnn_article_urls()
print("=== CNN URLs ===")
for url in cnn_urls:
    print(url)



=== CNN URLs ===


In [15]:
#NPR World
def get_npr_article_urls(section_url='https://www.npr.org/sections/news/'):
    headers = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(section_url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    links = set()
    for a in soup.find_all('a', href=True):
        href = a['href']
        if href.startswith('https://www.npr.org/20') and len(href.split('/')) > 5:
            links.add(href)

    return list(links)

npr_urls = get_npr_article_urls()
for url in npr_urls:
    print(url)
print(len(npr_urls))

https://www.npr.org/2025/05/05/nx-s1-5386679/met-gala-2025-preview-black-style
https://www.npr.org/2025/05/05/g-s1-64254/sean-diddy-combs-jury-selection-begins
https://www.npr.org/2025/05/04/g-s1-64239/does-a-president-need-to-uphold-the-constitution-trump-says-i-dont-know
https://www.npr.org/2025/05/05/nx-s1-5384963/marco-rubio-state-department-national-security-trump
https://www.npr.org/2025/05/04/nx-s1-5386456/mexico-president-rejected-trumps-plan-us-troops-across-border
https://www.npr.org/2025/05/04/nx-s1-5348841/ukraine-russian-occupied-luhansk-teen-escape
https://www.npr.org/2025/05/04/nx-s1-5386668/atf-trump-administration-gun-violence-memorial
https://www.npr.org/2025/05/05/nx-s1-5387239/trump-alcatraz-reopen
https://www.npr.org/2025/05/05/nx-s1-5386681/soviet-spacecraft-1972-reenter-earth-atmosphere
https://www.npr.org/2025/05/04/nx-s1-5386516/catholic-leaders-criticize-trump-ai-pope-photo
https://www.npr.org/2025/05/05/g-s1-64278/up-first-newsletter-israel-gaza-war-diddy-tri

In [18]:
#NYT WORLD
def get_nyt_article_urls():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)

    driver.get("https://www.nytimes.com/section/world")
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    article_urls = set()
    for a in soup.select('a'):
        href = a.get('href')
        if href and href.startswith("https://www.nytimes.com/202"):
            article_urls.add(href.split('?')[0])  # clean URL

    return list(article_urls)

nyt_urls = get_nyt_article_urls()
print("=== NYT URLs ===")
for url in nyt_urls:
    print(url)



=== NYT URLs ===
